# plotly

[https://plotly.com/python/](https://plotly.com/python/)

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

### Dane dotyczące Pokemonów

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/mini-pw/2021Z-DataVisualizationTechniques/master/labs/data/Pokemon.csv", index_col=0, sep=",")
df.head()

### Rysowanie prostych wykresów

In [ ]:
px.scatter(df,
           x = "Attack",
           y = "Defense",
           color = "Legendary",
           title = "Attack vs Defence",
           hover_name = "Name")

### Zmiana nazw osi/legendy

In [ ]:
px.scatter(df,
           x = "Attack",
           y = "Defense",
           color = "Legendary",
           title = "Atak vs. obrona",
           labels = {
               "Attack": "Punkty ataku",
               "Defense": "Punkty obrony",
               "Legendary": "Czy legendarny?"
           })

### Skale kolorów
- sekwencyjna
- dyskretna

In [ ]:
px.scatter(df,
           x = "Attack",
           y = "Defense",
           color = "HP",
           title = "Attack vs. Defense vs. HP",
           hover_name = "Name"
          ) # skala sekwencyjna

In [ ]:
px.scatter(df,
           x = "Attack",
           y = "Defense",
           color = "Type 1",
           title = "Attack vs. Defense by Type"
          ) # skala dyskretna

**Uwaga** domyślna skala kolorów ma tylko 10 kolorów, dla większej ilości typów następuje zapętlenie bez ostrzeżenia!

### Zmiana dyskretnej skali kolorów na jedną z wbudowanych

In [ ]:
px.scatter(df,
           x = "Attack",
           y = "Defense",
           color = "Type 1",
           title = "Attack vs. Defense by Type",
           color_discrete_sequence = px.colors.qualitative.Light24 # skala dyskretna ustawiona ręcznie
          ) 

Wbudowane dyskretne skale kolorów

In [ ]:
px.colors.qualitative.swatches()

In [ ]:
px.scatter(df,
           x = "Attack",
           y = "Defense",
           color = "HP",
           title = "Attack vs. Defense vs. HP",
           color_continuous_scale = px.colors.sequential.OrRd # skala sekwencyjna ustawiona ręcznie
          ) 

In [ ]:
px.colors.sequential.swatches_continuous()

In [ ]:
px.scatter(df,
           x = "Attack",
           y = "Defense",
           color = "Type 1",
           title = "Attack vs. Defense by Type",
           color_discrete_sequence = ["red", "blue", "#00ff00"] # możemy też skale ustawiać ręcznie
          ) 

### Inne typy wykresów
#### Histogram

In [ ]:
px.histogram(df,
             x = "HP",
             title = "Histogram of the HP variable")

#### Wykres słupkowy

In [ ]:
avg_df = df.groupby(["Type 1"]).agg({"HP": "mean", "Attack": "mean"}).reset_index().sort_values(["Attack", "HP"])

px.bar(avg_df,
       x = "Type 1",
       y = "Attack",
       title = "Average attack by Pokemon type",
       labels = {
           "Attack": "Average attack score",
           "Type 1": "Type of Pokemon"
       })

**Nowość!** Można podawać dane w postaci szerokiej

In [ ]:
px.bar(avg_df,
       x = "Type 1",
       y = ["Attack", "HP"],
       title = "Average attack and HP by Pokemon type",
       barmode = "group", ## domyślnie bez tej linijki słupki zestackowane - spróbuj zakomenotwać
       labels = {
           "Attack": "Average attack score",
           "Type 1": "Type of Pokemon",
           "variable": "Statistic",
           "value": "Value of statistic"
       })

#### Wykresy pudełkowe

In [ ]:
px.box(df,
       x = "Type 1",
       y = "Attack",
       color = "Type 1") #jeśli tu podamy inną zmienną np. "Stage" to każdy Typ zostanie jeszcze podzielony na Stage

#### Wykresy skrzypcowe

In [ ]:
fig = px.violin(df,
          y = "Type 1",
          x = "Attack",
          color = "Type 1",
          orientation = "h",
          box = True) # dodanie boxplotów w środku 
fig.show()

Czym tak naprawdę jest figura?

In [ ]:
print(fig)

#### Wykresy liniowe

- Możliwość podawania danych ręcznie (nie z pd.DataFrame)
- Zmiana zakresu osi

In [ ]:
x_list = [i for i in range(-10,11)]
y1_list = [i**2 for i in x_list]
y2_list = [i**3 for i in x_list]

fig = px.line(df,
              x = x_list,
              y = [y1_list, y2_list])
fig.show()

In [ ]:
fig.update_layout(xaxis_range=[0,10],
                  yaxis_range=[0, 100])
fig.show()

#### Heatmapy
- Za pomocą `px.imshow`

In [ ]:
correlation_df = df.iloc[:, 4:10].corr().round(3)
correlation_df

In [ ]:
px.imshow(correlation_df, 
          color_continuous_scale = px.colors.sequential.RdBu_r, # _r odwraca skalę kolorów
          title = "Correlation of Pokemon statistics",
          zmin = -1,
          zmax = 1,
          text_auto = True)

### Nakładanie wykresów
- oś logartmiczna

In [ ]:
fig = px.scatter(df,
           x = "Attack",
           y = "Defense",
           color = "HP",
           title = "Attack vs. Defense vs. HP",
           log_y = True # oś logarytmizna
          )

fig_line = px.line(df.groupby("Attack").agg("Defense").mean(), color_discrete_sequence = ["red"])

fig.add_trace(fig_line.data[0])
fig.update_layout(coloraxis_colorbar={"len":0.7}) # żeby legenda linii nie nachodziła na legendę kolorów

fig.show()

### Wykresy 3d
- zmiana markerów
- zmiana rozmiaru

In [ ]:
px.scatter_3d(df,
              x = "HP",
              y = "Defense",
              z = "Attack",
              color = "Type 1",
              symbol = "Type 1", # wzorki
              size = "Total",
              hover_name = "Name") # rozmiary

### Zaawansowane funkcje z użyciem `plotly.graph_objects`

Figury są tworzone w plotly za pomocą strukury danych `graph_objects`. Plotly express, pozwlala na szybkie tworzenie wykresów, jednak nie mamy wszystkich możliwości ich edycji. Niestety, nie działa to już tak fajnie z pandasem.

In [ ]:
import plotly.graph_objects as go

#### Zmiana template'u tekstu pojawiającego się po najechaniu

In [ ]:
fig = go.Figure()

obj1 = go.Scatter(x = df.Attack,
                  y = df.Defense,
                  mode = "markers",
                  marker = {"color": df["HP"]},
                  
                  text = [f"Ta kropka dotyczy pokemona {df.loc[i, 'Name']} o ataku {df.loc[i, 'Attack']}" for i in df.index],
                  
                  hovertemplate = """<i>%{text}</i><br>
                  <b>%{x:.2f}</b> 
                  <extra>A jego obrona to %{y:.1f}</extra>
                  """)

fig.add_trace(obj1)
fig.update_layout(title = "Attack vs. Defense vs. HP")
fig.update_xaxes(title = "Attack")
fig.update_yaxes(title = "Defense")

fig.show()



#### Wyłączenie guzików

In [ ]:
fig = go.Figure()

obj1 = go.Scatter(x = df.Attack,
                  y = df.Defense,
                  mode = "markers",
                  marker = {"color":df["HP"]})

fig.add_trace(obj1)
fig.update_layout(title = "Attack vs. Defense vs. HP")
fig.update_xaxes(title = "Attack")
fig.update_yaxes(title = "Defense")

fig.show(config = {"modeBarButtonsToRemove": ["toImage", "hoverClosestPie", "zoom2d", "pan2d", "select2d", "lasso2d", "zoomIn2d", "zoomOut2d", "autoScale2d", "resetScale2d"]})



In [ ]:
fig = go.Figure()

obj1 = go.Scatter(x = df.Attack,
                  y = df.Defense,
                  mode = "markers",
                  marker = {"color":df["HP"]})

fig.add_trace(obj1)
fig.update_layout(title = "Attack vs. Defense vs. HP")
fig.update_xaxes(title = "Attack")
fig.update_yaxes(title = "Defense")

fig.show(config = {"modeBarButtonsToRemove": ["toImage", "hoverClosestPie", "zoom2d", "pan2d", "select2d", "lasso2d", "zoomIn2d", "zoomOut2d", "autoScale2d", "resetScale2d"],
                    "displaylogo": False})



#### Wykres statyczny

In [ ]:
fig = go.Figure()

obj1 = go.Scatter(x = df.Attack,
                  y = df.Defense,
                  mode = "markers",
                  marker = {"color":df["HP"]})

fig.add_trace(obj1)
fig.update_layout(title = "Attack vs. Defense vs. HP")
fig.update_xaxes(title = "Attack")
fig.update_yaxes(title = "Defense")

fig.show(config = {"staticPlot":True})

#### Zmiana motywu wykresu

In [ ]:
#dostępne motywy
templates = ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']


for template in templates:
    fig = px.bar(avg_df,
       x = "Type 1",
       y = ["Attack", "HP"],
       title = "Average attack and HP by Pokemon type",
       barmode = "group",
       template = template, # podawanie motywu
       labels = {
           "Attack": "Average attack score",
           "Type 1": "Type of Pokemon",
           "variable": "Statistic",
           "value": "Value of statistic"
       })
    
    fig.show()

Stosowanie motywu do wszystkich wykresów

In [ ]:
import plotly.io as pio
pio.templates.default = "plotly_white"

## Zadania

Na podstawie zbioru danych wczytanego w komórce poniżej wykonaj następujące zadania.

1. Dobierz odpowiedni typ wizualizacji i narysuj wykres przedstawiający zależność ceny od objętości diamentu (przyjmij diament o kształcie prostopadłościanu o bokach x, y, z).

2. Narysuj wykres pokazujący rozkład ceny w zależności od kolorów diamentów (nałóż punkty obrazujące pojedyncze obserwacje).

3. Narysuj wykres przedstawiający liczbę diamentów o danej jakości cięcia (`cut`).

4. Narysuj wykres przedstawiający zależność liczby karatów od ceny, kolorem przedstaw kolor diamentu.


In [ ]:
diamonds_df = pd.read_csv("https://raw.githubusercontent.com/Swatikhedekar/Dada-Analysis-on-Diamonds-Dataset/main/diamonds.csv", index_col=0)
diamonds_df.head()